In [21]:
textgrid_dir = "/Users/joregan/Playing/textgrids_shared"
timecode_dir = "/Users/joregan/timecode_cut"
output_dir = "/tmp/textgrid_cut"

In [22]:
from pathlib import Path

textgrid_path = Path(textgrid_dir)
timecode_path = Path(timecode_dir)
output_path = Path(output_dir)

In [23]:
def get_timecode_offsets(filename):
    if type(filename) == Path:
        filename = str(filename)

    with open(filename) as inf:
        lines = [l.strip() for l in inf.readlines()]
        assert lines[0] == ",Frame,Time (Seconds),TimeCode", f"CSV file ({filename}) seems to be incorrect"
        p_start = lines[1].split(",")
        start = float(p_start[2])
        p_end = lines[-1].split(",")
        end = float(p_end[2])
        return start, end

In [40]:
from praatio import textgrid
from praatio.utilities.constants import Interval

def modify_textgrid(filename, start, end, output=None):
    if type(filename) == Path:
        filename = str(filename)
    if not output:
        output = filename
    elif type(output) == Path:
        output = str(output)

    tg = textgrid.openTextgrid(filename, True)
    new_tg = textgrid.Textgrid()
    for tiername in tg.tierNames:
        tier = tg.getTier(tiername)
        new_entries = []
        for entry in tier.entries:
            e_start = entry[0]
            e_end = entry[1]
            e_text = entry[2]

            new_start = e_start - start
            new_end = e_end - start

            if new_end < 0.0:
                continue
            elif e_start >= end and e_end > end:
                continue
            elif new_start < 0.0 and new_end > 0.0:
                if e_text != "":
                    print("Warning: truncating entry", filename, tiername, entry)
                if new_entries == []:
                    new_entries.append(Interval(0.0, new_end, e_text))
                else:
                    print("Shouldn't have existing entries!!", entry)
                    new_entries.append(Interval(0.0, new_end, e_text))
            elif e_start >= start and e_end <= end:
                new_entries.append(Interval(new_start, new_end, e_text))
            elif e_start <= end and e_end > end:
                if e_text != "":
                    print("Warning: truncating entry", filename, tiername, entry)
                new_entries.append(Interval(new_start, new_end, e_text))
            else:
                print("There should be no default case", entry)

        tier_start = new_entries[0][0]
        tier_end = new_entries[-1][1]
        new_tier = textgrid.IntervalTier(tiername, new_entries, tier_start, tier_end)
        new_tg.addTier(new_tier)

    new_tg.save(output, format="long_textgrid", includeBlankSpaces=True)